<a href="https://colab.research.google.com/github/julianl11/projektarbeit_1/blob/main/05_going_modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf data/

In [ ]:
import torch
from torch import nn

print(torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
import requests
from pathlib import Path
import zipfile

data_path = Path("data")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)
    print(f"{image_path} directory created.")

with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

In [ ]:
import os

def walk_through_dir(dir_path):
  """Walks through dir returning its contents."""
  print(f"dir_path = {dir_path}")
  for dirpath, dirnames, filenames in os.walk(dir_path):
    # print(f"dirnames: {dirnames}")
    # print(f"filenames: {filenames}")
    # print(f"dirpath: {dirpath}")
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

walk_through_dir(image_path)

In [ ]:
print(f"image_path: {image_path}")
train_dir = image_path / "train"
test_dir = image_path / "test"

In [ ]:
from torchvision import datasets
from torchvision import transforms

data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

train_data = datasets.ImageFolder(root=train_dir, transform=data_transform, target_transform=None)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform, target_transform=None)



In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_data, batch_size=1, shuffle=True, num_workers=1)
train_loader.dataset.classes

In [ ]:
train_data.classes

In [ ]:
import os
os.mkdir("going_modular")

In [ ]:
%%writefile going_modular/data_setup.py

"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""

import os

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

NUM_WORKERS = os.cpu_count()

def creat_dataloaders(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int = 32,
    num_workers: int=NUM_WORKERS
):
  """
  Creates training and testing DataLoaders.
  """

  train_data = datasets.ImageFolder(train_dir, transform=transform)
  test_data = datasets.ImageFolder(test_dir, transform=transform)

  train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
  test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

  class_names = train_data.classes

  return train_loader, test_loader, class_names


In [ ]:
from going_modular import data_setup

train_loader, test_loader, classe_names = data_setup.creat_dataloaders(train_dir, test_dir, transforms)
print(f"Train data loader: {train_loader}")
print(f"Test data loader: {test_loader}")
print(f"Class names: {classe_names}")

In [ ]:
%%writefile going_modular/model_builder.py
"""
Building a model in this file.
"""

import torch
import torch.nn as nn


class Tiny_vgg_custom(nn.Module):
  def __init__(self, input_shape:int, hidden_units: int, output_shape: int) -> None:
    super().__init__()

    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*30*30, out_features=output_shape)
    )

  def forward(self, x):
    x = self.conv_block_1(x)
    #print(x.shape)
    x = self.classifier(x)
    #print(x.shape)
    return x




In [ ]:
from going_modular import model_builder

model = model_builder.Tiny_vgg_custom(input_shape=3, hidden_units=10, output_shape=3)
model

In [ ]:
%%writefile going_modular/engine.py

"""
This file contains functions for training and testing a PyTorch model.
"""

from typing import Tuple


def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
  model.train()

  train_loss, train_acc = 0,0

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device, dtype=torch.float), y.to(device, dtype=torch.long)

    y_pred = model(X)

    loss = loss_fn(y_pred, y)
    train_loss += loss.item()

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

  with torch.no_grad():
    y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc += (y_pred_class == y).sum().item()/len(y_pred)






  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)

  return train_loss, train_acc


def test_step(model,
              dataloader,
              loss_fn, ):

  model.eval()

  test_loss, test_acc = 0,0

  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device, dtype=torch.float), y.to(device, dtype=torch.long)
      y_pred = model(X)

      loss = loss_fn(y_pred, y)
      test_loss += loss.item()

      y_pred_class = torch.argmax(y_pred, dim=1)
      test_acc += (y_pred_class == y).sum().item()/len(y_pred)

    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)

  return test_loss, test_acc

def train_and_evaluate(model, train_dataloader, test_dataloader, loss_fn, optimizer, epochs=3):

    results = {
        "train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }

    for epoch in range(epochs):
        train_loss, train_acc = train_step(model, dataloader=train_dataloader, loss_fn=loss_fn, optimizer=optimizer)
        test_loss, test_acc = test_step(model, dataloader=test_dataloader, loss_fn=loss_fn)
        print(f"Epoch: {epoch+1} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f} | Test acc: {test_acc:.4f}")

        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    return results



In [ ]:
%%writefile going_modular/utils.py

import torch
from pathlib import Path

def save_model(target_dir, model, model_name):

  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  print(f"Saving model to {target_dir_path}")
  model_path = target_dir_path / model_name
  torch.save(obj=model.state_dict(), f=model_path)

  return model_path

In [ ]:
from going_modular import utils

utils.save_model(target_dir="going_modular/models", model=model, model_name="05_going_modular_script_mode_tinyvgg_model.pth")